# Use python-pptx to generate powerpoint slides of data

In [ ]:
# Imports: 
import pathlib
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from tqdm.auto import tqdm

In [ ]:
# Define paths:
imagePaths = pathlib.Path('/Users/andrew/Library/CloudStorage/OneDrive-UCB-O365/research/data_analysis/rsoxs_suite/prsoxs_plots/2023C3')
linecutMapsPath = imagePaths.joinpath('linecut_maps_nitrogen_v1')
arMapsPath = imagePaths.joinpath('ar_maps_nitrogen_v1')
ISIsPath = imagePaths.joinpath('isi_nitrogen_v1')
moviePaths = imagePaths.joinpath('detector_movies_nitrogen_v1')

outPath = pathlib.Path('/Users/andrew/Library/CloudStorage/OneDrive-UCB-O365/research/presentations/update_slides')

In [ ]:
fname_list

In [ ]:
len([f.name for f in sorted(moviePaths.glob('*mp4'))][:])

In [ ]:
# IQ_sin_fl_heatmaps = set(linecutMapsPath.glob('sin-fl-sub*282-292*'))
IQ_sin_heatmaps = set(linecutMapsPath.glob('sin-sub*395-407*'))
IQ_all_heatmaps = set(linecutMapsPath.glob('*395-407*'))

IQ_heatmaps = IQ_all_heatmaps - IQ_sin_heatmaps

len([f.name for f in sorted(IQ_sin_heatmaps)][:])

In [ ]:
ar_polavg_maps = set(arMapsPath.glob('vset*polavg*'))
ar_all_maps = set(arMapsPath.glob('*'))

ar_pol090_maps = ar_all_maps - ar_polavg_maps

ar_polavg_doubled_maps = sorted(sorted(ar_polavg_maps) + sorted(ar_polavg_maps))

len([f.name for f in sorted(ar_polavg_doubled_maps)][:])

In [ ]:
ISIs_all = set(ISIsPath.glob('*'))
ISIs_sin = set(ISIsPath.glob('sin-sub*'))

ISIs = ISIs_all - ISIs_sin

len([f.name for f in sorted(ISIs_sin)])

In [ ]:
0 % 2

In [ ]:
# Generate slides:

prs = Presentation()

# Set slide dimensions for widescreen 16:9 aspect ratio
prs.slide_width = Inches(13.333)  # Width
prs.slide_height = Inches(7.5)    # Height

slide_layout = prs.slide_layouts[5]

for i, moviePath in enumerate(tqdm(sorted(moviePaths.glob('*mp4'))[:])):
    # Select appropriate path for each file
    poster_frame_path = moviePaths.joinpath(f'{moviePath.stem}.jpeg')
    fname_list = moviePath.name.split('_')
    pol_str = fname_list[-1][:-4]
    
    sample_name_list = fname_list[0:3]
    sample_name = '_'.join(sample_name_list)
    
    if pol_str == 'pol90deg':
        pol_orientation = 'Vertical'
    else:
        pol_orientation = 'Horizontal'
        
    # arMapPath = sorted(ar_pol090_maps)[i]
    arMapPath = sorted(ar_polavg_doubled_maps)[i]
    linecutMapPath = sorted(IQ_sin_heatmaps)[i]
    ISIPath = sorted(ISIs_sin)[i]
    
    # Add slide 
    slide = prs.slides.add_slide(slide_layout)

    # Format title
    title = slide.shapes.title
    title.left = Inches(0.22)
    title.top = Inches(0.4)
    title.width = Inches(4.8)
    title.height = Inches(2.91)
    title.text = f'Sample:\n{sample_name}\n\n{pol_orientation} X-ray Polarization'
    for paragraph in title.text_frame.paragraphs:
        paragraph.alignment = PP_ALIGN.LEFT
        paragraph.font.size = Pt(36)

    # Define image positions (these are just placeholders, adjust as necessary)
    left_movie = Inches(0)
    top_movie = Inches(3.41)
    height_movie = Inches(4.09)
    width_movie = Inches(5)

    left_linecutMap = Inches(5.08)
    top_linecutMap = Inches(3.5)
    height_linecutMap = Inches(3.75)

    left_arMap = Inches(5)
    top_arMap = Inches(0.25)
    height_arMap = Inches(3)    

    left_ISI = Inches(9.45)
    top_ISI = Inches(0.25)
    height_ISI = Inches(3)

    # Add images to the slide
    slide.shapes.add_movie(str(moviePath), left_movie, top_movie, height=height_movie, width=width_movie, poster_frame_image=str(poster_frame_path), mime_type='video/mp4')
    slide.shapes.add_picture(str(linecutMapPath), left_linecutMap, top_linecutMap, height=height_linecutMap)
    slide.shapes.add_picture(str(arMapPath), left_arMap, top_arMap, height=height_arMap)
    slide.shapes.add_picture(str(ISIPath), left_ISI, top_ISI, height=height_ISI)
            
        
prs.save(outPath.joinpath('2023C3_nitrogen_prsoxs_corr_auto-summary_v2.pptx'))
print('done!')

In [ ]:
for moviePath in tqdm(sorted(moviePaths.glob('*mp4'))):
    # Select appropriate path for each file
    poster_frame_path = moviePaths.joinpath(f'{moviePath.stem}.jpeg')
    fname_list = moviePath.name.split('_')
    pol_str = fname_list[-1][:-4]

    
#     if pol_str == 'pol0deg':
#         pol_orientation = 'Horizontal'
#     elif pol_str == 'pol90deg':
#         pol_orientation = 'Vertical'
        
    try:
        if len(fname_list)==4:
            sample_name = fname_list[0]
            # print(sample_name)
            linecutMapPath = list(linecutMapsPath.glob(f'{sample_name}_*pol0deg*q0.01-0.08*'))[0]
            arMapPath = list(arMapsPath.glob(f'vset*{sample_name}_*pol0deg*q-0.01-0.08*'))[0]
            ISIPath = list(ISIsPath.glob(f'{sample_name}_*qrange-0.01-0.08*pol0deg*'))[0]
        elif len(fname_list)==5:
            sample_name_list = fname_list[0:2]
            joined_sample_name = '_'.join(sample_name_list)     
            # print(joined_sample_name)
            linecutMapPath = list(linecutMapsPath.glob(f'{joined_sample_name}_*pol0deg*q0.01-0.08*'))[0]
            arMapPath = list(arMapsPath.glob(f'vset*{joined_sample_name}_*pol0deg*q-0.01-0.08*'))[0]
            ISIPath = list(ISIsPath.glob(f'{joined_sample_name}_*qrange-0.01-0.08*pol0deg*'))[0]
            
        print(linecutMapPath)
        print(arMapPath)
        print(ISIPath)
    except IndexError:
        pass
        
    # elif len(fname_list) == 7:
    #     sample_name_list = fname_list[0:4]
    #     joined_sample_name = '_'.join(sample_name_list)
    #     linecutMapPath = list(linecutMapsPath.glob(f'{joined_sample_name}*{pol_str}*'))[0]
    #     arMapPath = list(arMapsPath.glob(f'{joined_sample_name}*{pol_str}*'))[0]
    #     ISIPath = list(ISIsPath.glob(f'{joined_sample_name}*{pol_str}*'))[0]
    # elif len(fname_list) == 4:
    #     joined_sample_name = fname_list[0]
    #     linecutMapPath = list(linecutMapsPath.glob(f'{joined_sample_name}*{pol_str}*'))[0]
    #     arMapPath = list(arMapsPath.glob(f'{joined_sample_name}*{pol_str}*'))[0]
    #     ISIPath = list(ISIsPath.glob(f'{joined_sample_name}*{pol_str}*'))[0]

In [ ]:
[f.name for f in ISIsPath.glob(f'{sample_name}*')]

In [ ]:
sample_name

In [ ]:
# Generate slides:

prs = Presentation()

# Set slide dimensions for widescreen 16:9 aspect ratio
prs.slide_width = Inches(13.333)  # Width
prs.slide_height = Inches(7.5)    # Height

slide_layout = prs.slide_layouts[5]

for moviePath in tqdm(sorted(moviePaths.glob('*mp4'))):
    # Select appropriate path for each file
    poster_frame_path = moviePaths.joinpath(f'{moviePath.stem}.jpeg')
    fname_list = moviePath.name.split('_')
    pol_str = fname_list[-1][:-4]
    

    if pol_str == 'pol90deg':
        pol_orientation = 'Vertical'
    else:
        pol_orientation = 'Horizontal'
        
    try:
        if len(fname_list)==4:
            sample_name = fname_list[0]
            print(sample_name)
            linecutMapPath = list(linecutMapsPath.glob(f'{sample_name}_c*polavgdeg*q0.01-0.08*'))[0]
            arMapPath = list(arMapsPath.glob(f'vset*{sample_name}_c*polavgdeg*q-0.01-0.08*'))[0]
            ISIPath = list(ISIsPath.glob(f'{sample_name}_c*qrange-0.01-0.08*polavgdeg*'))[0]
        elif len(fname_list)==5:
            sample_name_list = fname_list[0:2]
            sample_name = '_'.join(sample_name_list)     
            print(sample_name)
            linecutMapPath = list(linecutMapsPath.glob(f'{sample_name}_*polavgdeg*q0.01-0.08*'))[0]
            arMapPath = list(arMapsPath.glob(f'vset*{sample_name}_*polavgdeg*q-0.01-0.08*'))[0]
            ISIPath = list(ISIsPath.glob(f'{sample_name}_*qrange-0.01-0.08*polavgdeg*'))[0]
            
        print(linecutMapPath)
        print(arMapPath)
        print(ISIPath)
        if pol_orientation == 'Horizontal':
            # Add slide 
            slide = prs.slides.add_slide(slide_layout)

            # Formate title
            title = slide.shapes.title
            title.left = Inches(0.22)
            title.top = Inches(0.4)
            title.width = Inches(4.8)
            title.height = Inches(2.91)
            title.text = f'Sample:\n{sample_name}\n\n{pol_orientation} X-ray Polarization'
            for paragraph in title.text_frame.paragraphs:
                paragraph.alignment = PP_ALIGN.LEFT
                paragraph.font.size = Pt(36)

            # Define image positions (these are just placeholders, adjust as necessary)
            left_movie = Inches(0)
            top_movie = Inches(3.41)
            height_movie = Inches(4.09)
            width_movie = Inches(5)

            left_linecutMap = Inches(5.08)
            top_linecutMap = Inches(3.5)
            height_linecutMap = Inches(3.75)

            left_arMap = Inches(5)
            top_arMap = Inches(0.25)
            height_arMap = Inches(3)    

            left_ISI = Inches(9.45)
            top_ISI = Inches(0.25)
            height_ISI = Inches(3)

            # Add images to the slide
            slide.shapes.add_movie(str(moviePath), left_movie, top_movie, height=height_movie, width=width_movie, poster_frame_image=str(poster_frame_path), mime_type='video/mp4')
            slide.shapes.add_picture(str(linecutMapPath), left_linecutMap, top_linecutMap, height=height_linecutMap)
            slide.shapes.add_picture(str(arMapPath), left_arMap, top_arMap, height=height_arMap)
            slide.shapes.add_picture(str(ISIPath), left_ISI, top_ISI, height=height_ISI)
            
    except IndexError:
        print('Only movie, skipping this slide')
        
        
prs.save(outPath.joinpath('2022C2_carbon_saxs_prsoxs_auto-summary_horizontal_v2.pptx'))